In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')  
import os
import json
from datetime import datetime

In [2]:
DATA_FOLDER = 'data'
OUTPUT_FOLDER = 'data/outputs'
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

GROUND_TRUTH_PATH = os.path.join(DATA_FOLDER, 'ground_truth.csv')
MARKERS_PATH = os.path.join(DATA_FOLDER, 'markers.csv')

In [3]:
results = {
    'status': 'processing',
    'timestamp': datetime.now().isoformat(),
    'errors': [],
    'ground_truth': {},
    'markers': {},
    'plots': []
}

In [ ]:
try:
    print("Loading ground truth data...")
    df_ground_truth = pd.read_csv(GROUND_TRUTH_PATH)
    
    results['ground_truth'] = {
        'shape': df_ground_truth.shape,
        'columns': list(df_ground_truth.columns),
        'head': df_ground_truth.head(10).to_dict('records'),
        'stats': {}
    }
    
    if 'PG' in df_ground_truth.columns:
        results['ground_truth']['stats'] = {
            'mean': float(df_ground_truth['PG'].mean()),
            'std': float(df_ground_truth['PG'].std()),
            'min': float(df_ground_truth['PG'].min()),
            'max': float(df_ground_truth['PG'].max()),
            'count': int(df_ground_truth['PG'].count())
        }
    
    print(f"✓ Loaded {df_ground_truth.shape[0]} rows")
    
except Exception as e:
    error_msg = f"Error loading ground truth: {str(e)}"
    print(error_msg)
    results['errors'].append(error_msg)
    df_ground_truth = None

Loading ground truth data...
✓ Loaded 148 rows


In [ ]:
try:
    print("Loading markers data...")
    df_markers = pd.read_csv(MARKERS_PATH)
    
    results['markers'] = {
        'shape': df_markers.shape,
        'columns': list(df_markers.columns),
        'head': df_markers.head(10).to_dict('records')
    }
    
    if 'condition' in df_markers.columns:
        results['markers']['conditions'] = df_markers['condition'].value_counts().to_dict()
    
    print(f"✓ Loaded {df_markers.shape[0]} rows")
    
except Exception as e:
    error_msg = f"Error loading markers: {str(e)}"
    print(error_msg)
    results['errors'].append(error_msg)
    df_markers = None

Loading markers data...
✓ Loaded 10 rows


In [ ]:
if df_ground_truth is not None and 'PG' in df_ground_truth.columns:
    try:
        print("Creating visualizations...")
        
        # Plot 1: PG signal over time
        plt.figure(figsize=(14, 5))
        plt.plot(range(len(df_ground_truth)), df_ground_truth['PG'], 
                linewidth=0.8, color='#2196F3', alpha=0.8)
        plt.xlabel('Sample Index', fontsize=12)
        plt.ylabel('PG Value', fontsize=12)
        plt.title('EmotiBit PG Signal Over Time', fontsize=14, fontweight='bold')
        plt.grid(True, alpha=0.3, linestyle='--')
        plt.tight_layout()
        
        plot1_path = os.path.join(OUTPUT_FOLDER, 'pg_timeseries.png')
        plt.savefig(plot1_path, dpi=100, bbox_inches='tight')
        plt.close()
        
        results['plots'].append({
            'name': 'PG Time Series',
            'path': plot1_path,
            'filename': 'pg_timeseries.png'
        })
        print(f"✓ Saved plot: {plot1_path}")
        
        # Plot 2: PG distribution histogram
        plt.figure(figsize=(10, 5))
        plt.hist(df_ground_truth['PG'], bins=50, color='#4CAF50', 
                alpha=0.7, edgecolor='black')
        plt.xlabel('PG Value', fontsize=12)
        plt.ylabel('Frequency', fontsize=12)
        plt.title('PG Value Distribution', fontsize=14, fontweight='bold')
        plt.grid(True, alpha=0.3, axis='y', linestyle='--')
        plt.tight_layout()
        
        plot2_path = os.path.join(OUTPUT_FOLDER, 'pg_histogram.png')
        plt.savefig(plot2_path, dpi=100, bbox_inches='tight')
        plt.close()
        
        results['plots'].append({
            'name': 'PG Distribution',
            'path': plot2_path,
            'filename': 'pg_histogram.png'
        })
        print(f"✓ Saved plot: {plot2_path}")
        
        # Plot 3: Rolling mean
        window = 100
        rolling_mean = df_ground_truth['PG'].rolling(window=window).mean()
        
        plt.figure(figsize=(14, 5))
        plt.plot(range(len(df_ground_truth)), df_ground_truth['PG'], 
                linewidth=0.5, color='#CCCCCC', alpha=0.5, label='Raw Signal')
        plt.plot(range(len(rolling_mean)), rolling_mean, 
                linewidth=2, color='#FF5722', label=f'{window}-point Moving Average')
        plt.xlabel('Sample Index', fontsize=12)
        plt.ylabel('PG Value', fontsize=12)
        plt.title('PG Signal with Moving Average', fontsize=14, fontweight='bold')
        plt.legend(fontsize=10)
        plt.grid(True, alpha=0.3, linestyle='--')
        plt.tight_layout()
        
        plot3_path = os.path.join(OUTPUT_FOLDER, 'pg_moving_average.png')
        plt.savefig(plot3_path, dpi=100, bbox_inches='tight')
        plt.close()
        
        results['plots'].append({
            'name': 'PG with Moving Average',
            'path': plot3_path,
            'filename': 'pg_moving_average.png'
        })
        print(f"✓ Saved plot: {plot3_path}")
        
    except Exception as e:
        error_msg = f"Error creating plots: {str(e)}"
        print(error_msg)
        results['errors'].append(error_msg)


Creating visualizations...
✓ Saved plot: data/outputs/pg_timeseries.png
✓ Saved plot: data/outputs/pg_histogram.png
✓ Saved plot: data/outputs/pg_moving_average.png


In [7]:
results['status'] = 'completed' if len(results['errors']) == 0 else 'completed_with_errors'
results_path = os.path.join(OUTPUT_FOLDER, 'results.json')

with open(results_path, 'w') as f:
    json.dump(results, f, indent=2)

print(f"\n✓ Analysis complete! Results saved to {results_path}")
print(f"Status: {results['status']}")
print(f"Plots generated: {len(results['plots'])}")



✓ Analysis complete! Results saved to data/outputs/results.json
Status: completed
Plots generated: 3
